In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_image_features_path = '../data/GOD/image_features_train.npy'
test_image_features_path = '../data/GOD/image_features.npy'

train_category_path = '../data/GOD/category_train.csv'
test_category_path = '../data/GOD/category_test.csv'

In [13]:
with open(train_category_path) as f:
    reader = csv.reader(f)
    train_cat = []
    for r in reader:
        train_cat += r
with open(test_category_path) as f:
    reader = csv.reader(f)
    test_cat = []
    for r in reader:
        test_cat += r
        
train_image_features = np.load(train_image_features_path)
test_image_features = np.load(test_image_features_path)

In [ ]:
train_cat_image_features_avg = []
for i in range(int(len(train_image_features)/8)):
    start = i * 8
    end = (i+1) * 8
    cat_avg = np.mean(train_image_features[start:end,:], axis=0, keepdims=True)
    train_cat_image_features_avg.append(cat_avg)
    
train_cat_image_features_avg = np.concatenate(train_cat_image_features_avg, axis=0)

In [ ]:
corr = np.corrcoef(train_cat_image_features_avg, test_image_features)
print(corr)

In [14]:
train_cat

['ostrich',
 'frog_1',
 'frog_2',
 'turtle',
 'rhinoceros',
 'snake',
 'trilobite',
 'scorpion',
 'spider',
 'centipede',
 'horseshoe_crab',
 'bird_1',
 'duck',
 'kangaroo',
 'cochlea',
 'basket_clam',
 'octopus',
 'crested_ibis',
 'bird_2',
 'penguin',
 'dolphin',
 'dog_1',
 'dog_2',
 'bear',
 'grasshopper',
 'cockroach',
 'praying_mantis',
 'starfish',
 'porcupine',
 'horse',
 'zebra',
 'deer',
 'giraffe',
 'skunk',
 'person',
 'gorilla',
 'chimpanzee',
 'elephant',
 'raccoon',
 'airship',
 'babycar',
 'bag',
 'man_with_bat',
 'baseball_mitt',
 'basketball_ring',
 'bustab',
 'lighthouse',
 'binocular',
 'water_bowl',
 'bowling',
 'boxing',
 'camera',
 'tire',
 'corn_flakes',
 'knife',
 'cup',
 'cd',
 'keyboard',
 'tv',
 'telephone',
 'floppy',
 'dumbbell',
 'earphone',
 'fighter',
 'fire_engine',
 'flashlight',
 'horn',
 'frisbee',
 'frying_pan',
 'gas_station',
 'radio',
 'golfball',
 'guitar_pick',
 'shoes',
 'calculator',
 'harmonica',
 'piano',
 'helicopter',
 'balloon',
 'pool',

In [15]:
test_cat

['Goldfish',
 'owl',
 'nan',
 'duck',
 'swan',
 'nan',
 'crab',
 'orca',
 'leopard',
 'Bat',
 'fly',
 'butterfly',
 'goat',
 'camel',
 'lama',
 'airplane',
 'nan',
 'beermug',
 'bowlingball',
 'bulldozer',
 'Cannon',
 'boat',
 'coffin',
 'carrier',
 'man_with_hat',
 'hat',
 'nan',
 'fire_extinguisher',
 'helmet',
 'piano',
 'grave',
 'man_with_hammock',
 'harp',
 'iPod',
 'knob',
 'post',
 'mandarin',
 'kitchen',
 'tower',
 'nan',
 'coin',
 'shredder',
 'Snowmobile',
 'socks',
 'Stained glass',
 'nan',
 'umbrella',
 'videoplayer',
 'washing_machine',
 'buttery?']